In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import gzip

import numpy as np
from IPython import display

import time

from keras import backend as K
from keras.callbacks import Callback
from keras.models import Model, load_model, model_from_json
from keras.utils import HDF5Matrix

from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow as tf

Using TensorFlow backend.


In [3]:
import sys
sys.path.append("..")

In [4]:
from utils.load_data import load_data
from utils.preprocess import multiple_batch_generator
from utils.comparams import calculate_auc, auc
from utils.callbacks import PlotCurves

W0610 23:56:24.192044 139846338945152 deprecation_wrapper.py:119] From ../utils/comparams.py:1: The name tf.local_variables_initializer is deprecated. Please use tf.compat.v1.local_variables_initializer instead.



# DATA

In [5]:
data_dir = '../data'

x_test, y_test_true = load_data(data_dir, purpose='test', is_gzip=True)#, norm='macenko')

# indexes
test_id = np.arange(len(x_test))

# create a useful dictionary structures
partition = {}
partition['test'] = test_id
    
test_labels = {str(i) : y_test_true[i].flatten()[0] for i in test_id}

len(test_labels)

32768

## MODEL

In [6]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! how many inputs were in the network? !!!!!!!!!!!!!!!!!!!!!!!!!
generator_num = 3

In [7]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'aug' : False
}

# Generators
test_generator = multiple_batch_generator(generator_num, list_IDs=partition['test'], 
                                          data=x_test, labels=test_labels, **params)

dependencies = {
    'auc': auc
}

In [8]:
network_filepath = '../Model/ensemble_3_model_bin_cross_arch_2_lr_0_0007_pure/ensemble_3_model_bin_cross_arch_2_lr_0_0007_pure_epoch_10.h5'

In [9]:
model = load_model(network_filepath, custom_objects=dependencies)

W0610 23:57:48.379984 139846338945152 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0610 23:57:48.461623 139846338945152 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0610 23:57:48.474153 139846338945152 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0610 23:57:48.474651 139846338945152 deprecation_wrapper.py:119] From /home/aorus/workspaces/simge/PatchCamelyon/.env/lib/python3.6/site-packages/keras/backend/

In [10]:
steps = int(len(test_labels)/batch_size)

In [16]:
preds = model.predict_generator(test_generator, steps=steps)

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv2_vgg16/Relu}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[main_output/Softmax/_20047]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv2_vgg16/Relu}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors ignore

In [ ]:
##### Calculate AUC 
true_labels = np.array(y_test_true).flatten()
pred_labels = np.array([p[1] for p in preds])
calculate_auc(true_labels, pred_labels)

In [15]:
del test_labels